In [1]:
from konlpy.tag import Twitter as OriginalTwitter

twitter_original = OriginalTwitter()
twitter_original.pos('우리아이오아이는 정말 이뻐요')

[('우리', 'Noun'),
 ('아이오', 'Noun'),
 ('아이', 'Noun'),
 ('는', 'Josa'),
 ('정말', 'Noun'),
 ('이뻐', 'Adjective'),
 ('요', 'Eomi')]

In [2]:
import sys
sys.path.append('../')

from customized_konlpy.tag import Twitter

twitter = Twitter()

twitter.add_dictionary('이', 'Modifier')
twitter.add_dictionary('우리', 'Modifier')
twitter.add_dictionary('이번', 'Modifier')
twitter.add_dictionary('아이오아이', 'Noun')
twitter.add_dictionary('행사', 'Noun')
twitter.add_dictionary('아이', 'Noun')
twitter.add_dictionary('번것', 'Noun')
twitter.add_dictionary('것', 'Noun')
twitter.add_dictionary('은', 'Josa')
twitter.add_dictionary('는', 'Josa')
twitter._customized_tagger.dictionary._pos2words

{'Josa': {'는', '은'},
 'Modifier': {'우리', '이', '이번'},
 'Noun': {'것', '번것', '아이', '아이오아이', '행사'}}

In [3]:
twitter.pos('우리아이오아이는 정말 이뻐요')

[('우리', 'Modifier'),
 ('아이오아이', 'Noun'),
 ('는', 'Josa'),
 ('정말', 'Noun'),
 ('이뻐', 'Adjective'),
 ('요', 'Eomi')]